In [ ]:
#  Deployed

In [ ]:
# bas concept eske articles ka url kaise find kia wah concept hai esmai,kaise url bnya articles ka wah concept hai esmai
# ALWAYS PREFER ESKA LOGIC, jaise search sey calue find kia but value to string ke form mai hota hai,so kaise split ke concept sey
# ek ek value ko acces kia n uss value ka use kia,,esmai wah concept hai

In [ ]:
#http://www.ifans.go.kr/knda/ifans/eng/main/IfansEngMain.do

In [ ]:
# 25-4-2021  ... done but not deployed  
# done done done 
#bas eska value dekhte rehna

In [2]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'ifans_go_kr_publication'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
unable_to_download_pdf = 0
publish_source = 'ifans.go.kr'
country = 'Korea'
language = 'English'
images_path = []

home_page = c.download_page('http://www.ifans.go.kr/knda/ifans/eng/main/IfansEngMain.do')
home_page = c.scrap('<button type="button">Publications</button></dt>(.*?)</ul>',home_page)

for _ in home_page.split('<li>')[1:]:
    cat_url = c.scrap('<a\s*href="(.*?)"', _)
    if 'http' not in cat_url:
        cat_url = 'http://www.ifans.go.kr'+ cat_url
   
    logger.info(f'Fetching cat url  {cat_url}\n')
    cat_page = c.download_page(cat_url)
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue 
        
    cat_page = c.scrap('class="board_list"(.*?)</ul>',cat_page)
    for i in cat_page.split('<li>')[1:]:
        
        # source_link
        source_number = c.scrap('<a\s*href=.*?onclick="fnCmdView\((.*?)\);', i)
        source_no = source_number.split(',')  # split is used beac in the source_number we are getting value in the string
        # i am getting a value in the form of string,so that y split the both the value,if there would be tuple or list then no need to perform split     
        source_link = f'http://www.ifans.go.kr/knda/ifans/eng/pblct/PblctView.do?csrfPreventionSalt=null&pblctDtaSn={source_no[0]}&menuCl={source_no[1]}&clCode={source_no[1]}' 
        source_link =re.sub("'",'',source_link)
        # handle duplicates
     
        
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        logger.info(f'Fetching {source_link}\n')
        
        page = c.download_page(source_link)  
        
        if page.startswith('Unable to fetch'):     
            logger.info(page)
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<strong\s*class="tit">(.*?)</strong>', page)
        source_headline = re.sub('&.*?;',"'",source_headline)
        source_headline = source_headline.strip()

     
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', current_time

        try:
            
            date_time_str = c.scrap('<span\s*class="date">.*?<em>(.*?)<', page)+'000000' 
            date_time_str = re.sub('[^\w+]','', date_time_str)  #
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%d%H%M%S')
            # Japan is 3 hours and 30 minutes ahead of India
            ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]

 
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    


        # source_content          
        source_content = c.scrap('div\s*class="editor\s*board_con con_in">(.*?)<br\s*/><br\s*/><br\s*/>\s*<strong>',page)
        if not source_content:
            source_content = c.scrap('div\s*class="editor\s*board_con con_in">(.*?)<dl\s*class="board_file">',page)
        source_content = re.sub('&.*?;',"",source_content)
        source_content = re.sub("''","",source_content)
        source_content = c.strip_html(source_content)

        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue

        
        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        
        # pdf
        pdf_path, pdf_name = '', ''
        pdf_url = c.scrap('<dl\s*class="board_file">.*?<a\s*href="(.*?)"',page)
        if 'http' not in pdf_url:
            pdf_url = 'http://www.ifans.go.kr' + pdf_url

        if pdf_url:
            pdf_name = c.scrap('uploadId=(.*?)\&',pdf_url) 
            if pdf_name == '':
                continue
            pdf_name = pdf_name + '.pdf'
            # pdf_path
            pdf_path = f'{pdf_directory}/{pdf_name}'        

            # download pdf
            pdf = c.download_pdf(pdf_url, pdf_path)
            if pdf.startswith('Unable to fetch'):
                logger.info(pdf) 
                unable_to_download_pdf += 1
                continue

      
        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                        "pdf_url": pdf_url,
                        "pdf_name": pdf_name,
                        "pdf_path":pdf_path
                    }

#         cl_data.insert_one(clientdata)  
        no_of_data += 1
logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 